In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import src



In [2]:
filename = 'data/creditcard.csv'
source = pd.read_csv(filename)

### Modelling Plan

0. Set up a testing framework
1. Set up a benchmark model on the data as it stands (Logistic Regression)
2. Apply preprocessings, namely normalisation, over/under sampling, outlier detection, and reapply to benchmark model.
3. Apply a range of models, including a RandomForest, NaiveBayes, XGBoost.
4. Concluding analysis

Disclaimer : This is the first time I have learnt about the imblearn library, so my main focus will be to understand its use rather than actually gaining an optimal area-under-precisison-recall curve.


### 0. Testing Framework

In the soruce scripts `src/tests.py`, the function `panel_of_tests(y_pred, y_observed)` conducts the folloing tests: 

* Precision
* Recall
* F1
* AUPRC (Average recall precision (Area under Recall Precision curve))
* Accuracy
* Balanced Accuracy
* ROC_AUC

Given the imbalanced dataset, we are most interested in AUPRC, but we can use the remaining metrics as a guide. Accuracy is there for completeness.

In [3]:
# an as example, demonstrate the testing panel function

y_obs = np.random.randint(0,2,10)
y_pred = np.random.randint(0,2,10)
y_score = np.random.uniform(0,1,10)

src.panel_of_tests(y_pred, y_obs, y_score)

{'Precision': 0.25,
 'Recall': 0.5,
 'F1': 0.3333333333333333,
 'AUPRC': 0.20833333333333331,
 'Accuracy': 0.6,
 'Balanced Accuracy': 0.5625,
 'ROC_AUC': 0.5625}

### 1. Set up benchmark model

In this secton we set up a very simply logistic regression model, via cross validation.

The code that performs the model implementation is found in `src/models.py`.

In [4]:
# Prepare the data for training

def prepare_data(data):
    Y = data['Class']
    X = data.drop(['Class'], axis = 1)

    X = X.values
    Y = Y.values
    return X, Y



In [7]:
# training data via cross validation

data = source.copy()
X,Y = prepare_data(data)

cv_model = src.cv_modelling(X,Y, folds = 5,sampling_function = None, outlier_function = None)
model = 'logistic'
C = '1'
model_string = ' '.join([model,C])
model_performancemodel_performance, models = cv_model.fit(model_string)
model_performance

 Fold [5/5]

,Precision,Recall,F1,AUPRC,Accuracy,Balanced Accuracy,ROC_AUC
0,0.846154,0.666667,0.745763,0.669032,0.99921,0.833228,0.833228
1,0.847222,0.616162,0.71345,0.63178,0.99914,0.807984,0.807984
2,0.859155,0.622449,0.721893,0.668897,0.999175,0.811137,0.811137
3,0.837838,0.632653,0.72093,0.647575,0.999157,0.816221,0.816221
4,0.828571,0.591837,0.690476,0.639212,0.999087,0.795813,0.795813
mean,0.843788,0.625953,0.718503,0.651299,0.999154,0.812876,0.812876


The table above provides the initial benchmark model - the interest of real interest is the AUPRC which is the area under the precision/recall curve. It is worth noting that the precision is consistently greater than recall, this is a result of the target imbalance and can be seen to come as follows, recall the formulae:

\begin{equation}\begin{aligned} \text{Precesion} = \frac{\text{TP}}{\text{TP}+\text{FP}},\\ \text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}} \end{aligned}.\end{equation}

Since, we have many more negative cases than positive, the model is more likely to misclassify positive (y=1) results as negative (y=0) than the other way around. This means that false negatives will tend to be relatively larger than false positives. This makes precision relatively larger then recall.

We can also use the value of the model coefficients to see the model importance of the features. This is since the logisitic model used above includes a lasso term.

In [8]:
# feature importances
Numerical = ['Time','V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']
       
def extract_importance(columns, feature_importances):
    idx = np.argsort(abs(feature_importances)).ravel()[::-1]
    return np.array(columns)[idx]

data = {'FOLD_' + str(i): extract_importance(Numerical, models[i].coef_) for i in range(0,5)}

df = pd.DataFrame(data =data)
df

,FOLD_0,FOLD_1,FOLD_2,FOLD_3,FOLD_4
0,V3,V3,V3,V3,V3
1,V15,V15,V15,V7,V7
2,V25,V7,V7,V15,V15
3,V14,V25,V25,V25,V25
4,V7,V14,V14,V14,V14
5,V22,V22,V22,V22,V22
6,V21,V9,V9,V11,V21
7,V11,V11,V11,V9,V11
8,V17,V21,V21,V21,V9
9,V9,V17,V8,V13,V17




Each fold found slightly different feature to be most important: we can find the intersection of the top 5. We can use a voting system amoung the folds to find the higher ranking features.

In [9]:
top = 5
[df.iloc[i].mode().values[0] for i in range(0,len(Numerical))][:top]

['V3', 'V15', 'V15', 'V25', 'V14']

### 2. Apply preprocessing, sampling techniques and outlier detection
In this subsection, we shall apply some basic preprocessing together with various sampling techniques, and outlier windowing to see how this changes the benchmark model.

In [8]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

#### 2.i : Normalise
Normalise the data with respect the median and the IQR, and check the benchmark model.

In [5]:
from sklearn.preprocessing import RobustScaler

def normalise(data):
    rs = RobustScaler()
    data[['Time','Amount']] = rs.fit_transform(data[['Time','Amount']].values)
    return data

In [11]:

data = source.copy()
data = normalise(data)
X,Y = prepare_data(data)



cv_model = src.cv_modelling(X,Y, folds = 5,sampling_function = None, outlier_function = None)
model = 'logistic'
C = '1'
model_string = ' '.join([model,C])
model_performancemodel_performance, models = cv_model.fit(model_string)
model_performance

 Fold [5/5]

,Precision,Recall,F1,AUPRC,Accuracy,Balanced Accuracy,ROC_AUC
0,0.846154,0.666667,0.745763,0.669032,0.99921,0.833228,0.833228
1,0.847222,0.616162,0.71345,0.63178,0.99914,0.807984,0.807984
2,0.859155,0.622449,0.721893,0.668897,0.999175,0.811137,0.811137
3,0.837838,0.632653,0.72093,0.647575,0.999157,0.816221,0.816221
4,0.828571,0.591837,0.690476,0.639212,0.999087,0.795813,0.795813
mean,0.843788,0.625953,0.718503,0.651299,0.999154,0.812876,0.812876


We see that the mean AUPRC has improved, thus this is the new accepted benchmark.

#### 2.ii Windowing

In this subsection, we consider windowing to remove any outliers - filtering for data above the 1% and below the 99% percentile per feature.



In [7]:
data = source.copy()
data = normalise(data)
X,Y = prepare_data(data)

outlier_function = src.delete_outliers

cv_model = src.cv_modelling(X,Y, folds = 5,sampling_function = None, outlier_function = outlier_function)
model = 'logistic'
C = '1'
model_string = ' '.join([model,C])
model_performance, models = cv_model.fit(model_string)
model_performance

 Fold [5/5]

,Precision,Recall,F1,AUPRC,Accuracy,Balanced Accuracy,ROC_AUC
0,0.195122,0.161616,0.176796,0.0624945,0.997384,0.580228,0.580228
1,0.375,0.181818,0.244898,0.153259,0.998051,0.590645,0.590645
2,0.240506,0.193878,0.214689,0.0994752,0.99756,0.596411,0.596411
3,0.410714,0.234694,0.298701,0.184775,0.998104,0.617057,0.617057
4,0.240964,0.204082,0.220994,0.0930842,0.997525,0.601487,0.601487
mean,0.292461,0.195217,0.231216,0.118618,0.997725,0.597166,0.597166


This appears to be a strong decrease in performance.

#### 2.ii Undersampling

In this subsection, we consider random undersampling. This is where we make the number of datapoints associated to the majority target class the same as the number of minority targets by simply randomly sampling the majority class.

In [10]:
# undersampling the data
under = RandomUnderSampler()

data = source.copy()
data = normalise(data)
X,Y = prepare_data(data)

sampling_function = under.fit_resample

cv_model = src.cv_modelling(X,Y,folds = 5, sampling_function = sampling_function, outlier_function = None)
model = 'logistic'
C = '1'
model_string = ' '.join([model,C])
model_performance, models = cv_model.fit(model_string)
model_performance

 Fold [5/5]

,Precision,Recall,F1,AUPRC,Accuracy,Balanced Accuracy,ROC_AUC
0,0.0412002,0.929293,0.0789022,0.714616,0.962291,0.945821,0.945821
1,0.0530077,0.89899,0.100112,0.698152,0.971911,0.935514,0.935514
2,0.0493827,0.897959,0.093617,0.488979,0.970085,0.934084,0.934084
3,0.0465473,0.928571,0.0886508,0.737661,0.967153,0.947895,0.947895
4,0.041919,0.918367,0.0801782,0.678554,0.963747,0.941096,0.941096
mean,0.0464114,0.914636,0.0882921,0.663592,0.967037,0.940882,0.940882


This actually slightly beats our very orginal benchmark model, whilst training very quickly since there is far less data.

The main problem with this approach is that it undergoes a very heavily information loss, from which we really cannot guarantee a clear out of sample generalisation. 

#### 2.iii Over sampling

We now consider oversampling

In [12]:
over = SMOTE()

data = source.copy()
data = normalise(data)
X,Y = prepare_data(data)

sampling_function = over.fit_resample

cv_model = src.cv_modelling(X,Y,folds = 5, sampling_function = sampling_function, outlier_function = None)
model = 'logistic'
C = '1'
model_string = ' '.join([model,C])
model_performance, models = cv_model.fit(model_string)
model_performance

 Fold [5/5]

,Precision,Recall,F1,AUPRC,Accuracy,Balanced Accuracy,ROC_AUC
0,0.0602981,0.89899,0.113016,0.72682,0.975475,0.937299,0.937299
1,0.0649057,0.868687,0.120787,0.660223,0.97802,0.923449,0.923449
2,0.0612903,0.969388,0.115291,0.772968,0.974404,0.9719,0.9719
3,0.0566625,0.928571,0.106808,0.753825,0.97328,0.950964,0.950964
4,0.0587828,0.867347,0.110104,0.743365,0.975878,0.921706,0.921706
mean,0.0603879,0.906597,0.113201,0.73144,0.975411,0.941064,0.941064


#### 2.iv Combination of over/under sampling

It is also posible to consider a combination of oversampling and then performing a random undersampling, as so:

In [13]:
class resample(object):
    def __init__(self, under_sample_stratergy, over_sample_strategy):
        self.under_sample = under_sample_stratergy
        self.over_sample = over_sample_strategy
        
        """
        Stratergy is such that count(y==1)/count(y == 0) = stratergy
        """
    
    def fit_resample(self, XX,YY):
        under = RandomUnderSampler(sampling_strategy=self.under_sample)
        over = SMOTE(sampling_strategy=self.over_sample)

        XX,YY = over.fit_resample(XX,YY)
        XX,YY = under.fit_resample(XX,YY)
        return XX,YY

In [14]:
over_under = resample(0.2,0.1)

data = source.copy()
data = normalise(data)
X,Y = prepare_data(data)

sampling_function = over_under.fit_resample

cv_model = src.cv_modelling(X,Y,folds = 5, sampling_function = sampling_function, outlier_function = None)
model = 'logistic'
C = '1'
model_string = ' '.join([model,C])
model_performance, models = cv_model.fit(model_string)
model_performance

 Fold [5/5]

,Precision,Recall,F1,AUPRC,Accuracy,Balanced Accuracy,ROC_AUC
0,0.222506,0.878788,0.355102,0.779062,0.994452,0.936721,0.936721
1,0.235294,0.888889,0.372093,0.781648,0.994786,0.94193,0.94193
2,0.194064,0.867347,0.317164,0.758888,0.993575,0.93057,0.93057
3,0.218837,0.806122,0.344227,0.683899,0.994716,0.900582,0.900582
4,0.241096,0.897959,0.38013,0.72428,0.994961,0.946544,0.946544
mean,0.222359,0.867821,0.353743,0.745556,0.994498,0.931269,0.931269


### 3. Grid search over many models

We now consider LogisticRegression, RandomForest, NaiveBayes, and XGBoost. We run these models over different sampling strategies and whether outlier removal occurs or not. To do this, we introduce a new function which allows us to amalgamate over and under sampling.

In [15]:
class resample(object):
    def __init__(self, under_sample_stratergy, over_sample_strategy):
        self.under_sample = under_sample_stratergy
        self.over_sample = over_sample_strategy
        
        """
        Stratergy is such that count(y==1)/count(y == 0) = stratergy
        """
    
    def fit_resample(self, XX,YY):
        if np.isnan(self.under_sample) and np.isnan(self.over_sample):
            return XX,YY
        elif not np.isnan(self.under_sample) and np.isnan(self.over_sample):
            under = RandomUnderSampler(sampling_strategy=self.under_sample)
            return under.fit_resample(XX,YY)
        elif np.isnan(self.under_sample) and not np.isnan(self.over_sample):
            over = SMOTE(sampling_strategy=self.over_sample)
            return over.fit_resample(XX,YY)
        else:
            
            under = RandomUnderSampler(sampling_strategy=self.under_sample)
            over = SMOTE(sampling_strategy=self.over_sample)

            XX,YY = over.fit_resample(XX,YY)
            XX,YY = under.fit_resample(XX,YY)
            return XX,YY

Next, we set up a results table:

In [16]:
def set_up_results_df():
    single_sampling_grid = np.linspace(0.2,1.0,4)
    over_under_grid = []
    for para_i in range(0,len(single_sampling_grid)):
        for para_j in range(para_i, len(single_sampling_grid)):
            over_under_grid.append((single_sampling_grid[para_i],single_sampling_grid[para_j]))
    over_under_grid = np.array(over_under_grid)

    results_df = pd.DataFrame(columns = ['OVER','UNDER','OUTLIER_REMOVAL','Precision','Recall', 'F1','AUPRC',  'Accuracy','Balanced Accuracy','ROC_AUC'], \
                      index = np.arange(1+2*len(single_sampling_grid) + len(over_under_grid)))

    results_df['OVER'][:len(single_sampling_grid)] = single_sampling_grid
    results_df['UNDER'][len(single_sampling_grid):2*len(single_sampling_grid)] = single_sampling_grid
    results_df['OVER'][2*len(single_sampling_grid):-1] = over_under_grid[:,0]
    results_df['UNDER'][2*len(single_sampling_grid):-1] = over_under_grid[:,1]

    with_outliers = results_df.copy()
    without_outliers = results_df.copy()

    without_outliers['OUTLIER_REMOVAL'] = 1
    with_outliers['OUTLIER_REMOVAL'] = 0

    results = pd.concat([with_outliers, without_outliers],0)
    results = results.reset_index()
    del results['index']


    metrics = ['Precision','Recall', 'F1','AUPRC', 'Accuracy','Balanced Accuracy','ROC_AUC']
    
    return results, metrics

As an example, let's see what the results table shows. Each row in the results table is a configuration to be considered for the grid search.



In [19]:
results, metrics = set_up_results_df()
results.iloc[0]

OVER                 0.2
UNDER                NaN
OUTLIER_REMOVAL        0
Precision            NaN
Recall               NaN
F1                   NaN
AUPRC                NaN
Accuracy             NaN
Balanced Accuracy    NaN
ROC_AUC              NaN
Name: 0, dtype: object

In [21]:
def run_specification(spec, model_string):
    """
    Runs the model against the specification determined by set_up_results_df().
    """
    sampler = resample(spec['UNDER'],spec['OVER'])
    if bool(spec['OUTLIER_REMOVAL']):
        cv_model = cv_modelling(X,Y,folds = 5, sampling_function = sampler.fit_resample,\
                                                    outlier_function = delete_outliers)
    else:
        cv_model = cv_modelling(X,Y,folds = 5, sampling_function = sampler.fit_resample,\
                                                    outlier_function = None)
    model_performance, models = cv_model.fit(model_string, verbose = False)
    return model_performance

#### 3.i Logistic Regression



In [ ]:
results, metrics = set_up_results_df()
n_configs, _ = results.shape

model_string = 'logistic 1'
for i in range(n_configs):
    print('\r', 'CONFIGURATIONS [{}/{}]'.format(i+1,n_configs), end='')
    spec = results.iloc[i]
    model_performance = run_specification(spec,model_string)
    results.at[i,metrics] = list(map(lambda x: model_performance.loc['mean'][x],metrics))
results.to_csv('results/Logistic.csv')

#### 3.ii Random Forest

In [ ]:
results, metrics = set_up_results_df()
n_configs, _ = results.shape

model_string = 'randomforest 20 4'
for i in range(n_configs):
    print('\r', 'CONFIGURATIONS [{}/{}]'.format(i+1,n_configs), end='')
    spec = results.iloc[i]
    model_performance = run_specification(spec,model_string)
    results.at[i,metrics] = list(map(lambda x: model_performance.loc['mean'][x],metrics))
results.to_csv('results/RandomForest.csv')

#### 3.iii NaiveBayes

In [ ]:
results, metrics = set_up_results_df()
n_configs, _ = results.shape

model_string = 'NBGaussian'
for i in range(n_configs):
    print('\r', 'CONFIGURATIONS [{}/{}]'.format(i+1,n_configs), end='')
    spec = results.iloc[i]
    model_performance = run_specification(spec,model_string)
    results.at[i,metrics] = list(map(lambda x: model_performance.loc['mean'][x],metrics))
results.to_csv('results/NaiveBayes.csv')

#### 3.iv XGBoost

In [ ]:
results, metrics = set_up_results_df()
n_configs, _ = results.shape

model_string = 'xgboost 20 4'
for i in range(n_configs):
    print('\r', 'CONFIGURATIONS [{}/{}]'.format(i+1,n_configs), end='')
    spec = results.iloc[i]
    model_performance = run_specification(spec,model_string)
    results.at[i,metrics] = list(map(lambda x: model_performance.loc['mean'][x],metrics))
results.to_csv('results/XGBoost.csv')

### 4. Model analysis

Now that we have collected all the results to the `results` folder, we can seek out the best configurations from the models studied so far.

In [28]:
logistic_result = 'results/Logistic.csv'
naivebayes_result = 'results/NaiveBayes.csv'
randomforest_results = 'results/RandomForest.csv'
xgboost_results = 'results/XGBoost.csv'

logistic_result = pd.read_csv(logistic_result, index_col = 0)
naivebayes_result = pd.read_csv(naivebayes_result, index_col = 0)
randomforest_results = pd.read_csv(randomforest_results, index_col = 0)
xgboost_results = pd.read_csv(xgboost_results, index_col = 0)

In [34]:
log_idx = logistic_result['AUPRC'].argmax()
logistic_result.iloc[log_idx]

OVER                      NaN
UNDER                     NaN
OUTLIER_REMOVAL      0.000000
Precision            0.871650
Recall               0.626036
F1                   0.727581
AUPRC                0.760470
Accuracy             0.999192
Balanced Accuracy    0.812937
ROC_AUC              0.812937
Name: 18, dtype: float64

In [35]:
nb_idx = naivebayes_result['AUPRC'].argmax()
naivebayes_result.iloc[nb_idx]

OVER                      NaN
UNDER                     NaN
OUTLIER_REMOVAL      1.000000
Precision            0.057466
Recall               0.412162
F1                   0.100321
AUPRC                0.140106
Accuracy             0.986893
Balanced Accuracy    0.700025
ROC_AUC              0.700025
Name: 37, dtype: float64

In [36]:
rf_idx = randomforest_results['AUPRC'].argmax()
randomforest_results.iloc[rf_idx]

OVER                      NaN
UNDER                     NaN
OUTLIER_REMOVAL      0.000000
Precision            0.911496
Recall               0.678932
F1                   0.777634
AUPRC                0.804665
Accuracy             0.999329
Balanced Accuracy    0.839408
ROC_AUC              0.839408
Name: 18, dtype: float64

In [37]:
xg_idx = xgboost_results['AUPRC'].argmax()
xgboost_results.iloc[xg_idx]

OVER                      NaN
UNDER                     NaN
OUTLIER_REMOVAL      0.000000
Precision            0.923132
Recall               0.792641
F1                   0.852086
AUPRC                0.815827
Accuracy             0.999526
Balanced Accuracy    0.896262
ROC_AUC              0.896262
Name: 18, dtype: float64